In [17]:
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL
from tabulate import tabulate
from urllib.parse import unquote

server_name   = "localhost"
database_name = "lab2"
username = 'sa'
password='reallyStrongPwd123'


connection_string = f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_name};DATABASE={database_name};UID={username};PWD={password}"
url_string = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

print('Connecting to database using URL string:')
unquoted_url = unquote(str(url_string))
print(unquoted_url, '\n')

try:    
    engine = create_engine(url_string)
    with engine.connect() as connection:
        print(f'Successfully connected to {database_name}!')
except Exception as e:
    print('Error while connecting to database:\n')
    print(e)

Connecting to database using URL string:
mssql+pyodbc://?odbc_connect=DRIVER=ODBC+Driver+17+for+SQL+Server;SERVER=localhost;DATABASE=lab2;UID=sa;PWD=reallyStrongPwd123 

Successfully connected to lab2!


In [38]:
def search_books(search_term, connection):
    query = text("""
    SELECT b.Title, inv.StoreID, st.StoreName, inv.Quantity
    FROM Books b
    JOIN Inventory inv ON b.ISBN13 = inv.ISBN
    JOIN Store st ON inv.StoreID = st.ID
    WHERE b.Title 
    LIKE :search_term
    """)
    results = connection.execute(query, {'search_term': f'%{search_term}%'}).fetchall()

    return results

search_term = input("Search for title: ").strip()

if search_term:
    with engine.connect() as connection:
        results = search_books(search_term, connection)
        for result in results:
            print(f"Book Title: {result.Title}, StoreID: {result.StoreID}, StoreName: {result.StoreName}, quantity: {result.Quantity}")
else:
    print("No search title written. Terminating program")


Book Title: To Kill a Mockingbird, StoreID: 1, StoreName: Book Haven, quantity: 20


In [39]:
from sqlalchemy import text

query2 = text("Select * from Author")
type(query2)

with engine.connect() as conn:
    result = conn.execute(query2)

    for row in result:
        print(f"{row.FirstName} {row.LastName}")

Harper Lee
George Orwell
Gabriel Garcia Marquez
Jane Austen
Oscar Wilde
Stephen King
Karin Boye


In [42]:
query3= text("Select StoreName, StoreAddress from Store")
type(query3)

with engine.connect() as conn:
    result = conn.execute(query3)

    for row in result:
        print(f"{row.StoreName} {row.StoreAddress}")

Book Haven 123 Main St
Page Turner Books 456 Elm St
The Reading Nook 789 Oak St
